# Algorithm for counting the components in directed graph

Here we consider components as distinct path, for an example like this:
                 
                                   L1->L2->L3
                                       L2->L4
                                       
We consider it has 2 components(distinct path):
                                    
                                   L1->L2->L3
                                   L1->L2->L4
       

## Initialize script

Setting up the environent and import data.
In this script I subset the links in NYU block and used trip numbers(instead error) as threshold.

In [5]:
import sys
import time
import pickle
#read python objects
sig_dist = pickle.load(open( "Signature_Distribution.p", "rb" ),encoding='latin1')
sig_links=list(sig_dist.keys())
Links_ps = pickle.load(open( "Links_per_Signature.p", "rb" ),encoding='latin1')
cm_path = pickle.load(open( "C_M_path.p", "rb" ),encoding='latin1')
#Persoanl directory,change it to your directory
dir_path="C://Users//Price//xueqiu//"

import pandas as pd
from matplotlib import cm
from matplotlib import colors 
#read csv files
links=pd.read_csv('C:\\Users\\price\\taxisim-master\\nyc_map4\\links.csv')
nodes=pd.read_csv("C:\\Users\\price\\taxisim-master\\nyc_map4\\nodes.csv")

import folium
from folium.features import (WmsTileLayer, RegularPolygonMarker, Vega, GeoJson,
                       CircleMarker, LatLngPopup,
                       ClickForMarker, TopoJson, PolyLine, MultiPolyLine,
                       )
from progress_bar import log_progress 
import numpy as np

In [6]:
import json
with open(dir_path+'full_link_trips.json') as json_data:
    o_D = json.load(json_data)
with open(dir_path+'full_link_travel_times.json') as json_data:
    o_T = json.load(json_data)
full_link_ids=np.loadtxt(dir_path+'full_link_ids.txt')
full_link_ids=list(map(int,full_link_ids))

In [7]:
nyu_links=[]
nyu_link_ids=[]
for i in range(len(full_link_ids)):
    coord=links[['startX','startY']][links['link_id']==full_link_ids[i]].values[0]    
    if (coord[1]>40.726) & (coord[1]<40.733) & (coord[0]<-73.99) & (coord[0]> -74.0):  
        nyu_links.append(i)
        nyu_link_ids.append(full_link_ids[i])

In [8]:
nyu_links=np.array(nyu_links)
nyu_link_ids=np.array(nyu_link_ids)

In [9]:
o_D=np.array(o_D)

In [10]:
nyu_T=o_D[:,nyu_links]
nyu_T=np.nan_to_num(nyu_T)
L=np.mean(nyu_T,axis=0)

## Get links with more than 50 trips 

In [11]:
L_table=pd.DataFrame({'L':L,'Link_ID':nyu_link_ids})

In [12]:
NY_COORDINATES = (40.7472,-73.98)
m = folium.Map(location=NY_COORDINATES, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(m)

#colmap={1:'#f46b42',2:'#f441f4',3:'#76f441',4:'#41f4e2',5:'#4146f4',6:'#dcf442',7:'#6841f4',8:'#41d9f4'}
test_links=[]

target_links=[]

L1=list(L_table[L_table['L']>50]['Link_ID'])

for i in log_progress(range(len(L1)),every=1):

        coords=links[['startY','startX','endY','endX']][links['link_id']==L1[i]].values[0]
        loc=[[coords[0],coords[1]],[coords[2],coords[3]]]
        test_links.append(i)
        node_ids=links[['begin_node_id','end_node_id']][links['link_id']==L1[i]].values[0]
        begin_node=node_ids[0]
        end_node=node_ids[1]
        popup = folium.Popup('Link_ID: '+str(L1[i])+' Begin_node: '+str(begin_node)+' End_node: '+str(end_node))
        #popup = folium.Popup('Index: '+str(i)+' Sig: '+str(signature)+' : '+str(final_H[:,i]))
        m.add_children(PolyLine(locations=loc,opacity=0.2,popup=popup))

In [13]:
m

# The algorithm

For any distinct path, its start link must belong to one of these two kinds:
    1. An entry link whose begin node is isolated from any other link. In other words, its begin node can not be any link's end node.
    
    
        L1->L2->L3->.....
                
        
    2. A link in a loop where we can not find an isolated entry link.
    
        L1->L2->L3->...->L1
       

## Step one:
 
For those paths start with an entry link, we can first subset all such entry links, take them as start points and traverse all possible path. This is the first step of our algorithm.

## Step two:

Fo the second situation, since the path starts in a loop, none of the link in a loop would be visitied
in step one, so we can remove all links visited by step one. Then we enumerate all links left to look for loops. Every time we find a loop, we should remove its links from the original set of links.

## Step three:

After Step one and step two, the only kind of path we have not visited is loop-with-braches:
        
        L1->L2->L3->L4->L5->L6->L7
                      ->L5->L1   

And we just enumerate loops we got in step two, take any link of the loop as start link, traverse all possible path, in the above example, starts from L3 we would get two path:

        L3->L4->L5->L6->L7(the branch)
        L3->L4->L5->L1->L2(the loop)

Since we already has the loop in step two, we would delete all loops we get in this step. For the braches, in this example it is incomplete,it should be:

        L1->L2->L3->L4->L5->L6->L7

So for all branches, we union the set of its links with the set of links in that loop.

## Step one

### Subset all entry links

In [14]:
temp_sub=links.iloc[(np.array(L1)-1),:]
end_node_set=set(list(temp_sub['end_node_id']))
entry_links=[]
for i in L1:
    if temp_sub[temp_sub['link_id']==i]['begin_node_id'].values[0] not in end_node_set:
        entry_links.append(i)
        print(i)

85360
118793
156998
204051
237377


### Traverse function

In [15]:

def path_from_one_point(start_link,path_ind):
    global global_ind

    """
    print('!!!!!!!!!')
    print(path_ind)
    print(start_link)
    print(global_ind)
    print('!!!!!!!!')
    """
    #print(global_ind)
    curr_link=start_link
    path[path_ind].append(start_link)
    curr_node=temp_sub[temp_sub['link_id']==start_link]['end_node_id'].values[0]
    #print(curr_node)
    next_links=temp_sub[temp_sub['begin_node_id']==curr_node]['link_id'].values
    #print(next_links)
    to_delete=[]
    for k in range(len(next_links)):
        if next_links[k] in set(path[path_ind]):
            to_delete.append(k)
    #next_links=np.array(list(map(int,next_links)))
    next_links=np.delete(next_links,np.array(to_delete))
    if len(next_links)==0:
        return
    elif len(next_links)==1:
        path_from_one_point(next_links[0],path_ind)
    else:
        temp_path=path[path_ind].copy()
        path_from_one_point(next_links[0],path_ind)
        
        for i in next_links[1:]:
            
            global_ind+=1
            #print(global_ind)
            path[global_ind]=temp_path
            path_from_one_point(i,global_ind)

### Implement traverse to all entry links

In [16]:
path={}
path[1]=[]
global_ind=1

i=0
while i<len(entry_links): 

    #print('g:')
    #print(global_ind)
    if i==0:
        path_from_one_point(entry_links[i],1)
    else:
        global_ind+=1
        path[global_ind]=[]
        path_from_one_point(entry_links[i],global_ind)
    i+=1

C:\Users\price\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


### Record the paths to one_way_path

In [17]:
one_way_path=path

In [18]:
one_way_path

{1: [85360],
 2: [118793,
  188335,
  117531,
  149421,
  119052,
  204078,
  204091,
  204103,
  204114,
  204128,
  204143,
  202367,
  143233,
  143854],
 3: [118793,
  188335,
  117531,
  149421,
  119052,
  204078,
  204091,
  204103,
  204114,
  204128,
  204143,
  204151,
  22989,
  103025],
 4: [118793,
  188335,
  117531,
  149421,
  119052,
  204078,
  204091,
  204103,
  204114,
  204128,
  204143,
  204151,
  22989,
  176421,
  176425],
 5: [118793, 188335, 117531, 149421, 227967, 163921, 79924, 112808],
 6: [118793, 188335, 117531, 151263],
 7: [118793, 190990, 202367, 143233, 143854],
 8: [118793, 190990, 204151, 22989, 103025],
 9: [118793, 190990, 204151, 22989, 176421, 176425],
 10: [156998,
  154143,
  62129,
  63784,
  99151,
  204091,
  204103,
  204114,
  204128,
  204143,
  202367,
  143233,
  143854],
 11: [156998,
  154143,
  62129,
  63784,
  99151,
  204091,
  204103,
  204114,
  204128,
  204143,
  204151,
  22989,
  103025],
 12: [156998,
  154143,
  62129,


### Get all visitied/unvisited links in step one 

In [19]:
visited_links=[]
def help_func(i):
    global visited_links
    visited_links=visited_links+i
list(map(help_func,list(one_way_path.values())))
visited_links=set(visited_links)
#visited_links

In [20]:
unvisited_links=[]
for i in L1:
    if temp_sub[temp_sub['link_id']==i]['link_id'].values[0] not in visited_links:
        unvisited_links.append(i)
        print(i)

43992
43995
45060
50473
52296
52769
52771
112248
113672
114151
145373
145374
157286
162933
173822
175581


## Step two

In [21]:
# This function is highly siilar to the traverse function we used in step one, but it is modified to record 
# links it found.
def loop_from_one_point(start_link,path_ind):
    global global_ind
    global loops
    """
    print('!!!!!!!!!')
    print(path_ind)
    print(start_link)
    print(global_ind)
    print('!!!!!!!!')
    """
    #print(global_ind)
    #print(loops)
    curr_link=start_link
    path[path_ind].append(start_link)
    curr_node=temp_sub[temp_sub['link_id']==start_link]['end_node_id'].values[0]
    #print(curr_node)
    next_links=temp_sub[temp_sub['begin_node_id']==curr_node]['link_id'].values
    #print(next_links)
    to_delete=[]
    for k in range(len(next_links)):
        if next_links[k] in set(path[path_ind]):
            if next_links[k]==path[path_ind][0]:
                if len(loops)<len(path[path_ind]):
                    loops=(path[path_ind]).copy()
                    #print('update')
                    #print(loops)
                    #print('update')
            to_delete.append(k)
    #next_links=np.array(list(map(int,next_links)))
    next_links=np.delete(next_links,np.array(to_delete))
    if len(next_links)==0:
        return
    elif len(next_links)==1:
        loop_from_one_point(next_links[0],path_ind)
    else:
        temp_path=path[path_ind].copy()
        loop_from_one_point(next_links[0],path_ind)
        
        for i in next_links[1:]:
            
            global_ind+=1
            #print(global_ind)
            path[global_ind]=temp_path
            loop_from_one_point(i,global_ind)

## Implement step two

Note that here every time we find a loop, we will add its links into visited_links set and get new unvisited links.

In [22]:
n=0
loops=[]
loop_links=[]
len_ori_unvisited=len(unvisited_links)
while (len(unvisited_links)>0):
    global loop_links
    global unvisited_loops
    path={}
    path[1]=[]
    global_ind=1
    loops=[]
    loop_from_one_point(unvisited_links[0],1)
    visited_links.add(unvisited_links[0])
    #print('unvisited:')
    #print(unvisited_links[0])

    
    if loops != []:
        loop_links.append(loops)
    for i in loops:
        visited_links.add(i)

    unvisited_links=[]
    for i in L1:
        if temp_sub[temp_sub['link_id']==i]['link_id'].values[0] not in visited_links:
            unvisited_links.append(i)
            print(i)
    n+=1

43995
45060
50473
52296
52769
52771
145373
145374
175581
45060
50473
52296
52769
52771
145373
145374
175581
50473
52296
52769
52771
145373
145374
175581
52296
52769
52771
145373
145374
175581
52769
52771
145373
145374
175581
175581


C:\Users\price\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [23]:
#loop_links

[[43992, 157286, 114151, 173822, 112248, 113672, 162933],
 [52769, 145374, 52771, 145373]]

## Step three

In step three, we delete all loops and union the set of loop-braches links with the set of links in that loop.

In [24]:
path={}

global_ind=0
for i in loop_links:
    start=global_ind+1
    path[global_ind+1]=[]
    path_from_one_point(i[0],global_ind+1)
    for j in range(start,global_ind+1):
        if set(path[j])==set(i):
            del path[j]
        else:
            path[j]=list(set(path[j]).union(set(i)))
         
    #print('i:    '+ str(i))
#path

i:    [43992, 157286, 114151, 173822, 112248, 113672, 162933]
i:    [52769, 145374, 52771, 145373]


C:\Users\price\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


### Store path in loop_branch 

In [25]:
loop_branch_path=path

In [27]:
#loop_branch_path

## Visually check the components

By changing L1 below.

In [28]:
NY_COORDINATES = (40.7472,-73.98)
m = folium.Map(location=NY_COORDINATES, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(m)

#colmap={1:'#f46b42',2:'#f441f4',3:'#76f441',4:'#41f4e2',5:'#4146f4',6:'#dcf442',7:'#6841f4',8:'#41d9f4'}
test_links=[]



L1=loop_branch_path[29]

for i in log_progress(range(len(L1)),every=1):

        coords=links[['startY','startX','endY','endX']][links['link_id']==L1[i]].values[0]
        loc=[[coords[0],coords[1]],[coords[2],coords[3]]]
        test_links.append(i)
        node_ids=links[['begin_node_id','end_node_id']][links['link_id']==L1[i]].values[0]
        begin_node=node_ids[0]
        end_node=node_ids[1]
        popup = folium.Popup('Link_ID: '+str(L1[i])+' Begin_node: '+str(begin_node)+' End_node: '+str(end_node))
        #popup = folium.Popup('Index: '+str(i)+' Sig: '+str(signature)+' : '+str(final_H[:,i]))
        m.add_children(PolyLine(locations=loc,opacity=0.3,popup=popup))

In [29]:
m

In [229]:
links[links['link_id']==43992]

,link_id,begin_node_id,end_node_id,begin_angle,end_angle,street_length,osm_name,osm_class,osm_way_id,startX,startY,endX,endY,osm_changeset,birth_timestamp,death_timestamp
43991,43992,1773084405,42437626,14.89,-164.66,80.185,TheBowery,primary,165777951,-73.991723,40.726363,-73.991603,40.727114,11772076,1405267167641,40000000000000
